# Import necessary packages

In [ ]:
from pyimagesearch.transform import four_point_transform
from imutils import perspective
from keras.preprocessing import image
from skimage.filters import threshold_local
import cv2
import imutils
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Extract Image Block

In [ ]:
Path = r'C:\Users\hedho\Desktop\Photo OCR\Receipt\R.jpg'        

In [ ]:
def ExtractImageBlock(Path):

    # load the image and compute the ratio of the old height
    # to the new height, clone it, and resize it

    img = cv2.imread(Path)
    ratio = img.shape[0] / 500.0
    orig = img.copy()
    img = imutils.resize(img, height = 500)
 
    # convert the image to grayscale, blur it, and find edges
    # in the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)

    # find the contours in the edged image, keeping only the
    # largest ones, and initialize the screen contour
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
 
    blk = []
    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:
            screenCnt = approx
        break

    imgblk = perspective.four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
    
    return imgblk

# Extract Text Block

In [ ]:
image = ExtractImageBlock(Path)

In [ ]:
def ExtractTextBlock(image):
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    morph = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))
    _,binary = cv2.threshold(morph,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    morphex = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (80, 5)))
    
    _, contours, _ = cv2.findContours(morphex,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    con = cv2.drawContours(img.copy(), contours, -1, (0, 255, 0), 2)

    cord = []
    for contour in contours:
        # get rectangle bounding contour
        [x, y, w, h] = cv2.boundingRect(contour)
        
        if (w<10) or (h<10):
            continue
    
        cord.append([max(0,x-10), max(0,y-10), w+10, h+10])
        
        # draw rectangle around contour on original image
        cv2.rectangle(img.copy(), (x, y), (x + w, y + h), (0, 255, 0), 3)
        
    ocr = []
    for i in range(1,len(cord)):
        x, y, w, h = cord[-i]
        text = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
        crop_img = text[max(0,y):y+h, max(0,x):x+w]
   
    return ocr

In [ ]:
for i in ocr:
    plt.figure(figsize=(10,20))
    plt.imshow(i, cmap = 'gray')